In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import random

In [2]:
df = pd.read_csv('/directory/file_name.csv')

### Identify high quality HC time series

In [3]:
df_HC = df[df['LABEL']=='Control']
df_has_data_sum = df_HC.groupby('ID')['has_data'].agg(['sum','count'])
df_has_data_sum['percentage_has_data_missing'] = 1 - df_has_data_sum['sum'] / df_has_data_sum['count']  # Calculate what % of overall data is missing for each participant

df_has_gps_data_sum = df_HC.groupby('ID')['has_gps_data'].agg(['sum','count'])
df_has_gps_data_sum['percentage_has_gps_data_missing'] = 1 - df_has_gps_data_sum['sum'] / df_has_gps_data_sum['count']  # Calculate what % of GPS data is missing for each participant

df_percent_missing = pd.DataFrame(index=df_has_data_sum.index)
df_percent_missing = pd.concat([df_percent_missing, df_has_data_sum['percentage_has_data_missing'], df_has_gps_data_sum['percentage_has_gps_data_missing']], axis=1)

df_segments_to_be_included = df_percent_missing.drop(df_percent_missing[df_percent_missing['percentage_has_data_missing'] > 0.1].index)  # Select participants for inclusion in training set
segments_to_be_included = list(df_segments_to_be_included.index)

### Assign all good quality HCs to training for now, assign other groups to testing

In [4]:
df['Testing1_Training0'] = 1
df.loc[df['ID'].isin(segments_to_be_included), 'Testing1_Training0'] = 0

### Withhold some good quality HCs for testing

In [5]:
random.seed(14)
move_HC_to_testing = random.sample(segments_to_be_included,15)
df.loc[df['ID'].isin(move_HC_to_testing), 'Testing1_Training0'] = 1

In [6]:
df.to_csv('/directory/file_name.csv', index=False)